In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/e-coli-genomes/E coli dataset/genes_GCF_000352125.1.tsv
/kaggle/input/e-coli-genomes/E coli dataset/genes_GCF_900457215.1.tsv
/kaggle/input/e-coli-genomes/E coli dataset/genes_GCF_013186525.1.tsv
/kaggle/input/e-coli-genomes/E coli dataset/genes_GCF_002965625.1.tsv
/kaggle/input/e-coli-genomes/E coli dataset/genes_GCF_003697165.2 (1).tsv
/kaggle/input/e-coli-genomes/E coli dataset/genes_GCF_002993265.1.tsv
/kaggle/input/e-coli-genomes/E coli dataset/genes_GCF_022972535.1.tsv
/kaggle/input/e-coli-genomes/E coli dataset/genes_GCF_900499355.1.tsv
/kaggle/input/e-coli-genomes/E coli dataset/genes_GCF_910664935.1.tsv
/kaggle/input/e-coli-genomes/E coli dataset/genes_GCF_023276905.1.tsv
/kaggle/input/e-coli-genomes/E coli dataset/genes_GCF_000690815.1.tsv
/kaggle/input/e-coli-genomes/E coli dataset/genes_GCF_009899265.1.tsv
/kaggle/input/e-coli-genomes/E coli dataset/genes_GCF_001912235.1.tsv
/kaggle/input/e-coli-genomes/E coli dataset/genes_GCF_019269175.1.tsv
/kaggle/input/e-

# Installation

In [2]:
#Installation
!pip install -q -U google-generativeai
%pip install -qU 'langgraph==0.2.45' 'langchain-google-genai==2.0.4'


import google.generativeai as genai
from kaggle_secrets import UserSecretsClient

#Set API key and generate model
GOOGLE_API_KEY = UserSecretsClient().get_secret("GEMINI_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-bigquery 2.34.4 requires packaging<22.0dev,>=14.3, but you have packaging 24.2 which is incompatible.
jupyterlab 4.2.5 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
kfp 2.5.0 requires google-cloud-storage<3,>=2.2.1, but you have google-cloud-storage 1.44.0 which is incompatible.
kfp 2.5.0 requires requests-toolbelt<1,>=0.8.0, but you have requests-toolbelt 1.0.0 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
thinc 8.3.2 requires numpy<2.1.0,>=2.0.0; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
ydata-profiling 4.10.0 requires scipy<1.14,>=1.4.1, but you have scipy 

# Preprocessing

In [3]:
import glob

directory_path = '/kaggle/input/e-coli-genomes/E coli dataset'

df = pd.concat((pd.read_csv(f, delimiter='\t') for f in glob.glob(os.path.join(directory_path, '*.tsv'))), ignore_index=True)
df.head()

,scaffoldId,begin,end,strand,locusTag,proteinId,desc
0,NZ_KB732999.1,205,1176,-,WE1_RS22575,WP_000817028.1,ParB/RepB/Spo0J family plasmid partition protein
1,NZ_KB732999.1,1176,2342,-,WE1_RS22570,WP_001238646.1,plasmid-partitioning protein SopA
2,NZ_KB732999.1,2642,3339,-,WE1_RS0122580,WP_094096600.1,IS1-like element IS1A family transposase
3,NZ_KB732999.1,3397,3885,+,WE1_RS0122590,NaN,recombinase family protein (pseudogene)
4,NZ_KB732999.1,4210,5952,-,WE1_RS22565,WP_001523375.1,ATP-binding cassette domain-containing protein


In [4]:
import pandas as pd
import os


#df["left_neighbor_id"] = df["proteinId"].shift(1)
#df["right_neighbor_id"] = df["proteinId"].shift(-1)
df["left_neighbor_desc"] = df["desc"].shift(1)
df["right_neighbor_desc"] = df["desc"].shift(-1)
sampleddf = df.sample(1700)
droppeddf = sampleddf.drop(columns=["scaffoldId", "locusTag", "strand", "begin", "end"], axis=1)
droppeddf.to_csv("/kaggle/working/sample.csv", index=False)
droppeddf.head()



,proteinId,desc,left_neighbor_desc,right_neighbor_desc
90235,WP_012775982.1,hypothetical protein,DUF968 domain-containing protein,protein Rem
10026,WP_174246215.1,MFS transporter,IS6-like element IS26 family transposase,IS3-like element IS600 family transposase (pse...
77194,WP_000638141.1,AAA family ATPase,RNA ligase family protein,S-ribosylhomocysteine lyase
55605,WP_105267144.1,glutathione-regulated potassium-efflux system ...,type 3 dihydrofolate reductase,glutathione-regulated potassium-efflux system ...
20803,WP_000633583.1,pirin family protein,protein YhaL,DNA-binding transcriptional regulator YhaJ


In [5]:
import glob
import sys
import json
#df = pd.concat((pd.read_csv(f) for f in glob.glob(os.path.join('/kaggle/working/', '*.csv'))), ignore_index=True)
#onesetdf = pd.read_csv('/kaggle/working/sample.csv')
'''

json_data = droppeddf.to_json(orient="records")

with open('/kaggle/working/oneset.json', 'w') as file:
    json.dump(json_data, file)
size = sys.getsizeof(json_data)
print(f"Size of the JSON object: {size/1000000} mb")'''

size = sys.getsizeof(droppeddf)
print(f"Size of the JSON object: {size/1000000} mb")

Size of the JSON object: 0.593526 mb


# Defining the state object

In [6]:
from typing import Annotated, List, Dict, Any
from typing_extensions import TypedDict
import csv

from langgraph.graph.message import add_messages

class ConversationState(TypedDict): #object state in the form of a dictionary that's passed around to the nodes. Keeps track of conversation history
    messages: Annotated[list, add_messages]
        
    sample_csv: List[Dict[str, Any]]
    
    gene: str #the gene in question that the user is asking about?
        
    finished: bool

In [7]:
def upload_to_gemini(path, mime_type=None):
  """Uploads the given file to Gemini.

  See https://ai.google.dev/gemini-api/docs/prompting_with_media
  """
  file = genai.upload_file(path, mime_type=mime_type)
  print(f"Uploaded file '{file.display_name}' as: {file.uri}")
  return file

def wait_for_files_active(files):
  """Waits for the given files to be active.

  Some files uploaded to the Gemini API need to be processed before they can be
  used as prompt inputs. The status can be seen by querying the file's "state"
  field.

  This implementation uses a simple blocking polling loop. Production code
  should probably employ a more sophisticated approach.
  """
  print("Waiting for file processing...")
  for name in (file.name for file in files):
    file = genai.get_file(name)
    while file.state.name == "PROCESSING":
      print(".", end="", flush=True)
      time.sleep(10)
      file = genai.get_file(name)
    if file.state.name != "ACTIVE":
      raise Exception(f"File {file.name} failed to process")
  print("...all files ready")
  print()

In [8]:
files = [
  #upload_to_gemini('/kaggle/working/sample.csv', mime_type="text/plain"),
]

#wait_for_files_active(files)


In [9]:
#where did these descs come from.
#note these are only limited to coding regions of DNA
INSTRUCTIONS = ("system",
"""You are given genes from the E coli genome that have been classified in the Genome Taxonomy Database.This dataset you are given
have columns as follows:

'proteinID': Unique identifier given to every protein in the NCBI.
'desc': the function the gene encodes for.
'left_neighbor_desc': The description of the left neighbor of the gene in question.
'right_neighbor_desc': The description of the right neighbor of the gene in question.

A user is going to give you a putative or hypothetical gene and the description of its left and right neighbor. Your job is to determine what the
genes encodes for, based on the examples given to you. Give the user the reasoning behind your conclusion, and cite specific examples from the dataset. 
If the user asks follow-up questions, limit your response to a succinct paragraph. If the user goes off-topic, redirect them back to the original question.
""")


welcome_message="""Hello! My expertise is in the E coli genome. If you type in a putative/unidentified gene, but know the function of its neighbors, I can tell you
what I think is it's function. Type q to quit. How can I help?
"""

INSTRUCTIONS = INSTRUCTIONS[:1] + (INSTRUCTIONS[1] + droppeddf.to_string(),)



In [10]:
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI

# Create the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest",
  generation_config=generation_config)



# Adding a human node

In [11]:
from langchain_core.messages.ai import AIMessage
from typing import Literal

def chatbot(state: ConversationState) -> ConversationState:
    message_history = [INSTRUCTIONS] + state["messages"]
    return {"messages": [model.invoke(message_history)]}

def human_node(state: ConversationState) -> ConversationState:
    last_message = state["messages"][-1]
    print("\n", "Model: ", last_message.content)
    
    user_input = input("You: ")
    
    if user_input in {"q", "quit"}:
        state["finished"] = True
    
    return state | {"messages": [("user", user_input)]}


def chatbot_with_welcome_message(state: ConversationState) -> ConversationState:
    if state["messages"]: 
        new_output = model.invoke([INSTRUCTIONS] + state["messages"])
    else:
        new_output = AIMessage(content=welcome_message)
    return state | {"messages": [new_output]}

def exit_human_node(state: ConversationState) -> Literal["chatbot", "__end__"]:
    if state.get("finished", False):
        return END
    else:
        return "chatbot"


graph_builder = StateGraph(ConversationState)
graph_builder.add_node("chatbot", chatbot_with_welcome_message)
graph_builder.add_node("human", human_node)
graph_builder.add_edge(START, "chatbot")

graph_builder.add_edge("chatbot", "human")

graph_builder.add_conditional_edges("human", exit_human_node)

chat_graph = graph_builder.compile()

#uncomment to start chat
#state = chat_graph.invoke({"messages": []})


In [12]:
#response = chat_session.send_message(prompt)
#print(response.text)